In [12]:
import sys
import os
import logging
sys.path.append(r"..")
from utils import *
import numpy as np
import pandas as pd
import deepctr
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,recall_score
from gensim import models
from scipy import sparse
from scipy.sparse import csr_matrix, coo_matrix
import gc
from collections import defaultdict
from collections import Counter
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'n_estimators': 5000,
    'metric': 'mae',
    'learning_rate': 0.01,
    'min_child_samples': 5,
    'min_child_weight': 0.01,
    'subsample_freq': 1,
    'num_leaves': 31,
    'max_depth': -1,
    'subsample': 0.8,
    'colsample_bytree': 0.6,
    'reg_alpha': 0,
    'reg_lambda': 5,
    'verbose': -1,
    'random_state': 4590,
    'n_jobs': 4,
}


In [13]:
#user_ids = pd.read_pickle(f"{pickle_path}/user_ids_relencode.pickle")
#user_ids = pd.read_pickle(f"{pickle_path}/user_ids_ctr.pickle")

#user = pd.read_csv(train_preliminary_p + "user.csv", encoding='utf-8')
#user_ids = user_ids.merge(user, how='left', on='user_id')
#save_pickle(user_ids, f"{pickle_path}/train_user_ids.pickle")
user_ids = pd.read_pickle(f"{pickle_path}/train_user_ids.pickle") #保存从8min缩小到3min
#del user
gc.collect()
print(user_ids.shape)
print(user_ids.columns)
user_ids.head(2)

(1900000, 32)
Index(['user_id', 'creative_id', 'ad_id', 'product_id', 'product_category',
       'advertiser_id', 'industry', 'click_times', 'time', 'creative_id_times',
       'ad_id_times', 'product_id_times', 'advertiser_id_times',
       'product_category_times', 'industry_times', 'creative_id_len',
       'ad_id_len', 'product_id_len', 'product_category_len', 'advertiser_len',
       'industry_len', 'mean_clicktimes', 'max_clicktimes', 'min_clicktimes',
       'mean_time', 'max_time', 'min_time', 'time_len', 'time_clicktimes',
       'time_creativeids', 'age', 'gender'],
      dtype='object')


,user_id,creative_id,ad_id,product_id,product_category,advertiser_id,industry,click_times,time,creative_id_times,...,max_clicktimes,min_clicktimes,mean_time,max_time,min_time,time_len,time_clicktimes,time_creativeids,age,gender
0,1,"[642275, 170988, 685687, 1307085, 100774, 5953...","[581171, 156554, 620126, 1166887, 92441, 55685...","[1, 125, 1, 1, 1132, 1, 1, 1, 1, 1244, 55, 55,...","[5, 2, 5, 5, 2, 18, 5, 5, 18, 2, 2, 2, 2]","[6411, 8522, 26781, 12996, 10113, 13007, 15341...","[323, 7, 104, 323, 1, 323, 73, 214, 64, 235, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]","[20, 20, 20, 39, 40, 43, 46, 52, 60, 64, 64, 7...","[3872964, 1123872, 3935014, 706993, 226871, 37...",...,2,1,47.461538,76,20,10,"[3, 1, 1, 1, 1, 1, 1, 2, 1, 2]","[3, 1, 1, 1, 1, 1, 1, 2, 1, 1]",4.0,1.0
1,2,"[52634, 128035, 32734, 478744, 10372, 349730, ...","[49388, 117016, 31581, 435462, 11432, 318236, ...","[76, 69, 118, 118, 1193, 76, 1, 1064, 1, 1, 10...","[2, 2, 2, 2, 2, 2, 18, 2, 18, 18, 2, 18, 2, 18...","[20681, 9438, 16619, 23545, 705, 20681, 31504,...","[315, 235, 7, 7, 314, 315, 47, 7, 47, 47, 239,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10, 11, 14, 17, 28, 28, 28, 38, 38, 39, 41, 4...","[3670932, 575824, 3013599, 3644043, 316325, 34...",...,2,1,53.822222,90,10,28,"[1, 1, 1, 1, 3, 2, 1, 1, 3, 2, 1, 4, 1, 4, 1, ...","[1, 1, 1, 1, 3, 2, 1, 1, 3, 2, 1, 4, 1, 4, 1, ...",10.0,1.0


In [14]:
## 计算idf值 creative_id
import itertools
from collections import Counter
def fea_idf(fea):
    fea_id_lists = user_ids[fea].map(lambda x: list(set(x)))
    fea_l = list(itertools.chain(*fea_id_lists))
    save_pickle(Counter(fea_l), pickle_path+f'{fea}_idf')
    
fea_idf('creative_id')  
fea_idf('ad_id')  
fea_idf('product_id')  
fea_idf('product_category')  
fea_idf('advertiser_id')  
fea_idf('industry')  
fea_idf('click_times')  

In [15]:
def tfidf_matrix_p(features_list, min_dfs):
    for fea, min_df in zip(features_list, min_dfs):
        tfidfer = TfidfVectorizer(analyzer='word', token_pattern=u"(?u)\\b\\w+\\b", min_df=1, max_features=min_df)  # (?u)\b\w+\b"，这样就不会忽略单个的字符
        if fea == 'creative_id':
            tfidf_matrix = tfidfer.fit_transform(user_ids[fea].map(lambda x:' '.join([str(i) for i in x])).values)
            save_pickle(tfidfer.vocabulary_, pickle_path+f'creative_id_tfidf_v_{min_df}')
            save_pickle(tfidf_matrix, pickle_path+f'creative_id_tfidf_m_{min_df}')

            print(f'tfidf matrix {fea} size:{tfidf_matrix.shape}')
        else:
            tfidf_matrix_fea = tfidfer.fit_transform(user_ids[fea].map(lambda x:' '.join([str(i) for i in x])).values)
            print(f'tfidf matrix {fea} size:{tfidf_matrix_fea.shape}')
            tfidf_matrix = csr_matrix(sparse.hstack((tfidf_matrix, tfidf_matrix_fea)))
            save_pickle(tfidfer.vocabulary_, pickle_path+f'{fea}_tfidf_v_{min_df}')
            save_pickle(tfidf_matrix_fea, pickle_path+f'{fea}_tfidf_m_{min_df}')
    
    save_pickle(tfidf_matrix, pickle_path+f'tfidf_m')
   
    print(f'tfidf matrix size:{tfidf_matrix.shape}')
    return tfidf_matrix

In [16]:
tfidf_features = ['creative_id', 'ad_id', 'product_id', 'product_category', 'advertiser_id', 'industry', 'click_times',
                 'creative_id_times', 'ad_id_times',  'product_id_times', 'advertiser_id_times',
                 'product_category_times', 'industry_times',]
tfidf_features = ['creative_id', 'ad_id', 'product_id', 'product_category', 'advertiser_id', 'industry', 'click_times',
                 ]
max_features = [10000, 10000, 300000, 300000, 300000, 300000, 300000]
tfidf_matrix = tfidf_matrix_p(tfidf_features, max_features)

tfidf matrix creative_id size:(1900000, 10000)
tfidf matrix ad_id size:(1900000, 10000)
tfidf matrix product_id size:(1900000, 39057)
tfidf matrix product_category size:(1900000, 18)
tfidf matrix advertiser_id size:(1900000, 57870)
tfidf matrix industry size:(1900000, 332)
tfidf matrix click_times size:(1900000, 8)
tfidf matrix size:(1900000, 117285)


In [17]:
offline = False
if offline:
    train_filter = 700000
else:
    train_filter = 900000


评估提交

0.861665 0.280975 1.14264  
0.90131 0.31792 1.21923  min_df=5  
0.909795 0.35963 1.269425  min_df=25    (1900000, 262005)  
0.908905 0.367675 1.27658  min_df=50  (1900000, 138555)  
0.905905 0.36898 1.2748849999999998  100 (1900000, 71208)  
0.909795 0.36898 1.278775  线上1.285234  25和100的结合    


0.91127 0.38208 1.29335



(0.922435,  0.418795  
0.92264  0.41987  

In [18]:
def lr_pred(pred_y, n_classes):
    cv = True
    model_type = 'lgb'
    seed = 2020
    train_filter = 900000
    train_x = tfidf_matrix[0:train_filter]
    train_y = user_ids[pred_y][0:train_filter]
    test_x =  tfidf_matrix[900000:]
    if cv:
        n_fold = 5
        count_fold = 0
        preds_list = list()
        oof = np.zeros((train_x.shape[0], n_classes))
        kfolder = KFold(n_splits=n_fold, shuffle=True, random_state=seed)
        kfold = kfolder.split(train_x, train_y)
        for train_index, vali_index in kfold:
            print("training......fold",count_fold)
            count_fold = count_fold + 1
            k_x_train = train_x[train_index]
            k_y_train = train_y[train_index]
            k_x_vali = train_x[vali_index]
            k_y_vali = train_y[vali_index]
            if model_type == 'lgb':
                
                reg = LogisticRegression(solver='lbfgs', multi_class='multinomial', n_jobs=-1)
                reg_model = reg.fit(k_x_train, k_y_train)
                k_pred = reg_model.predict_proba(k_x_vali)
                pred = reg_model.predict_proba(test_x)
               
    #             lgb_model = lgb_model.fit(k_x_train, k_y_train, eval_set=[(k_x_vali, k_y_vali)],
    #                                       early_stopping_rounds=200, verbose=False, eval_metric="mae",
    #                                       )
    #             k_pred = lgb_model.predict(k_x_vali, num_iteration=lgb_model.best_iteration_)
    #             pred = lgb_model.predict(test_x, num_iteration=lgb_model.best_iteration_)
            preds_list.append(pred)
            oof[vali_index] = k_pred
#     preds_columns = ['preds_{id}'.format(id=i) for i in range(n_fold)]
#     preds_df = pd.DataFrame(data=preds_list)
#     preds_df = preds_df.T
#     preds_df.columns = preds_columns
#     preds = list(preds_df.mean(axis=1))
    return preds_list, oof

In [19]:
gender_prob, gender_oof = lr_pred('gender',2)
age_prob, age_oof = lr_pred('age',10)


training......fold 0
training......fold 1
training......fold 2
training......fold 3
training......fold 4
training......fold 0
training......fold 1
training......fold 2
training......fold 3
training......fold 4


In [20]:
#gender_tune_weight = search_weight(user_ids['gender'][:900000]-1, gender_oof, init_weight=[1.0]*2,class_num=2, step=0.001)
#age_tune_weight = search_weight(user_ids['age'][:900000]-1, age_oof, init_weight=[1.0]*10,class_num=10, step=0.001)

gender_oof_pre = np.argmax(gender_oof,axis=1) + 1
age_oof_pre = np.argmax(age_oof,axis=1) + 1
gender_accuracy = accuracy_score(user_ids['gender'][:900000], gender_oof_pre)
age_accuracy = accuracy_score(user_ids['age'][:900000], age_oof_pre)

gender_accuracy,age_accuracy,gender_accuracy+age_accuracy

(0.9200077777777778, 0.39763444444444446, 1.3176422222222222)

In [21]:
gender_prob_mean = np.array(gender_prob).mean(axis=0)
gender_pre = np.argmax(gender_prob_mean,axis=1) + 1

age_prob_mean = np.array(age_prob).mean(axis=0)
age_pre = np.argmax(age_prob_mean,axis=1) + 1

np.save(f"{sub_path}/lr_gender_prob.npy", gender_prob_mean)
np.save(f"{sub_path}/lr_age_prob.npy", age_prob_mean)

gender_pre.shape, age_pre.shape

((1000000,), (1000000,))

In [22]:
sub = pd.DataFrame()
sub['user_id'] = range(3000001,4000001)
sub['predicted_age'] = age_pre
sub['predicted_gender'] = gender_pre
sub.to_csv(f"{sub_path}/submission.csv", index=False, encoding='utf-8')
print(sub.shape)
sub.head()

(1000000, 3)


,user_id,predicted_age,predicted_gender
0,3000001,3,1
1,3000002,7,2
2,3000003,7,1
3,3000004,3,1
4,3000005,4,1


0.9222855555555556 0.42004555555555556 1.3423311111111111  
线上1.3472